In [1]:
import pandas as pd
fname = 'C:/Users/huangm/Desktop/sqlResult_1558435.csv'
data = pd.read_csv(fname, encoding='gb18030')
data = data.fillna('') #将NaN转换为''
data.head()

,id,author,source,content,feature,title,url
0,89617,,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


## 正负样本量不平衡

In [2]:
data_sampled = data.sample(n=3000)
data_pos = data_sampled[data_sampled['source'] == '新华社']
data_neg = data_sampled[data_sampled['source'] != '新华社']
len(data_pos), len(data_neg)

(2665, 335)

## 直接选取样本，使之平衡（分类器设置class_weight也可以）

In [9]:
data_pos = data[data['source'] == '新华社']
data_neg = data[data['source'] != '新华社']

N_samples = 8000
data_pos = data_pos.sample(n = int(N_samples / 2))
data_neg = data_neg.sample(n = int(N_samples / 2))

data_sample = data_pos.append(data_neg)
len(data_sample)

8000

## 数据预处理

In [10]:
import jieba
import re

def cut(string):
    """切词"""
    return ' '.join(jieba.cut(string))

def token(string):
    """匹配文字"""
    return re.findall(r'[\d\w]+', string)

In [11]:
content = [cut(''.join(token(content))) for content in data_sample['content'].tolist()]
y = [1 if source == '新华社' else 0 for source in data_sample['source'].tolist()]
new_data = pd.DataFrame([{'content':content_i,'y':y_i} for content_i, y_i in zip(content, y)])
new_data.head()

,content,y
0,新华社 照片 外代 2017 年 6 月 4 日 n 外代 二线 足球 欧冠 皇马 卫冕 成...,1
1,新华社 西安 6 月 3 日电 记者 蔺娟 记者 3 日 从 陕西省 人力资源 和 社会保障...,1
2,新华社 照片 外代 2017 年 4 月 16 日 n 外代 二线 法国 博 瓦尔 动物园 ...,1
3,n 新华社 上海 ５ 月 ２ 7 日电 题大 数据 提示 这个 端午节 你 怎么 过 n 新...,1
4,新华社 照片 陵水 海南 2017 年 5 月 28 日 n 体育 3 篮球 2017 中国...,1


## TFIDF向量化

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer # sklearn tfidf向量化
import numpy as np

vectorized = TfidfVectorizer(max_features=8000)
news = new_data['content'].tolist()
news_vectors = vectorized.fit_transform(news)
news_vectors.shape, len(y)

((8000, 8000), 8000)

In [15]:
news_array = news_vectors.toarray()

## 数据集分割

In [16]:
from sklearn.model_selection import train_test_split # 划分数据集

# 划分数据集
X_train, X_test, Y_train, Y_test = train_test_split(news_array, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, len(Y_train), len(Y_test)

((6400, 8000), (1600, 8000), 6400, 1600)

## 构建模型

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

def get_performance(y_valid, y_pred, y_pred_prob):
    """获得模型表现"""
    # precision 查准率
    print('precision score:{}'.format(precision_score(y_valid, y_pred)))
    # recall 召回率
    print('recall score:{}'.format(recall_score(y_valid, y_pred)))
    # f1 score用来综合precision和recall作为一个评价指标
    print('f1 score:{}'.format(f1_score(y_valid, y_pred)))
    # roc_auc_score 判断二分类预测模型优劣的标准
    print('roc_auc_score:{}'.format(roc_auc_score(y_valid, y_pred_prob[:,1])))

In [18]:
# 将训练集数据划分为训练和验证集
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.2, random_state=52)

In [19]:
# 假定在一个k分类问题中，验证集中共有n个样本
# predict返回的是n个元素的一维数组，每个代表这每个样本得预测值
# predict_proba返回是n行k列的二维数组，每一行代表每个样本在k个类别预测的概率，相加为1

def get_prediction(model, x_valid, y_valid):
    """
    获得模型预测结果
    """
    print('score={}'.format(model.score(x_valid, y_valid)))
    y_pred = model.predict(x_valid)
    y_pred_prob = model.predict_proba(x_valid)
    return y_pred, y_pred_prob

### KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier # KNN分类器

def knn(n_neighbors, n_jobs, weights, x_train, y_train):
    """
    n_neighbors 选择邻近点个数
    n_jobs设定工作的core数量,为-1代表全部的CPU核进行工作
    weights: 'uniform' or 'distance'
    """
    knn_model = KNeighborsClassifier(n_neighbors=n_neighbors, n_jobs= n_jobs)
    knn_model.fit(x_train, y_train)
    return knn_model

#### ① n_neighbors = 3    
#### ② weights = 'uniform'

In [21]:
knn_model_1 = knn(3, 2, 'uniform', x_train, y_train)
y_pred, y_pred_prob = get_prediction(knn_model_1, x_valid, y_valid)
get_performance(y_valid, y_pred, y_pred_prob)

score=0.68671875
precision score:0.9645669291338582
recall score:0.38461538461538464
f1 score:0.5499438832772167
roc_auc_score:0.7606331682092624


#### ① n_neighbors = 5    
#### ② weights = 'uniform'

In [22]:
# 使用上面划分好的训练集和测试集，保证前后数据一致

knn_model_2 = knn(5, 2, 'uniform', x_train, y_train)
y_pred, y_pred_prob = get_prediction(knn_model_2, x_valid, y_valid)
get_performance(y_valid, y_pred, y_pred_prob)

score=0.66484375
precision score:0.9814814814814815
recall score:0.3328100470957614
f1 score:0.4970691676436108
roc_auc_score:0.7605513793027678


#### ① n_neighbors = 7   
#### ② weights = 'uniform'

In [23]:
# 使用上面划分好的训练集和测试集，保证前后数据一致

knn_model_3 = knn(7, 2, 'uniform', x_train, y_train)
y_pred, y_pred_prob = get_prediction(knn_model_3, x_valid, y_valid)
get_performance(y_valid, y_pred, y_pred_prob)

score=0.6484375
precision score:0.9947089947089947
recall score:0.29513343799058084
f1 score:0.4552058111380145
roc_auc_score:0.7600142581257888


#### ① n_neighbors = 7
#### ② weights = '‘distance’'

In [24]:
# 使用上面划分好的训练集和测试集，保证前后数据一致

knn_model_4 = knn(7, 2, 'distance', x_train, y_train)
y_pred, y_pred_prob = get_prediction(knn_model_4, x_valid, y_valid)
get_performance(y_valid, y_pred, y_pred_prob)

score=0.6484375
precision score:0.9947089947089947
recall score:0.29513343799058084
f1 score:0.4552058111380145
roc_auc_score:0.7600142581257888


### Gaussian Naive Bayes

In [25]:
from sklearn.naive_bayes import GaussianNB

def nb(x_train, y_train):
    """高斯朴素贝叶斯"""
    NB_model = GaussianNB()
    NB_model.fit(x_train, y_train)
    return NB_model

In [27]:
# 使用上面划分好的训练集和测试集，保证前后数据一致

NB_model = nb(x_train, y_train)
y_pred, y_pred_prob = get_prediction(NB_model, x_valid, y_valid)
get_performance(y_valid, y_pred, y_pred_prob)

score=0.778125
precision score:0.7313237221494102
recall score:0.8759811616954474
f1 score:0.797142857142857
roc_auc_score:0.7806189589126715


### Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

def lr(x_train, y_train):
    """逻辑回归"""
    lr_model = LogisticRegression(n_jobs=2)
    lr_model.fit(x_train, y_train)
    return lr_model

In [29]:
# 使用上面划分好的训练集和测试集，保证前后数据一致

lr_model = lr(x_train, y_train)
y_pred, y_pred_prob = get_prediction(lr_model, x_valid, y_valid)
get_performance(y_valid, y_pred, y_pred_prob)

F:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
F:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


score=0.93359375
precision score:0.9876325088339223
recall score:0.8775510204081632
f1 score:0.9293433083956775
roc_auc_score:0.9910691397027767


### SVM

In [30]:
from sklearn import svm

def sv(x_train, y_train, C=1, kernel='rbf', gamma='auto',class_weight=None, probability=True):
    """SVM"""
    sv_model = svm.SVC(C=C, kernel=kernel, gamma=gamma, class_weight=class_weight, probability=probability)
    sv_model.fit(x_train, y_train)
    return sv_model

```
C=1
kernel = 'linear'
gamma='auto'
class_weight=None
```

In [ ]:
sv_model = sv(x_train, y_train, C=1, kernel='linear')
y_pred, y_pred_prob = get_prediction(sv_model, x_valid, y_valid)
get_performance(y_valid, y_pred, y_pred_prob)

In [ ]:
sv_model = sv(x_train, y_train, C=100, kernel='linear')
y_pred, y_pred_prob = get_prediction(sv_model, x_valid, y_valid)
get_performance(y_valid, y_pred, y_pred_prob)

In [ ]:
sv_model = sv(x_train, y_train, C=500, kernel='linear')
y_pred, y_pred_prob = get_prediction(sv_model, x_valid, y_valid)
get_performance(y_valid, y_pred, y_pred_prob)

### Random Forest

In [32]:
from sklearn.ensemble import RandomForestClassifier

def rf(x_train, y_train, n_estimators=10, max_depth=None, random_state=None):
    """Random Forest"""
    rf_model = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    rf_model.fit(x_train, y_train)
    return rf_model

In [33]:
rf_model = rf(x_train, y_train, n_estimators=100, max_depth=None, random_state=None)
y_pred, y_pred_prob = get_prediction(rf_model, x_valid, y_valid)
get_performance(y_valid, y_pred, y_pred_prob)

score=0.9
precision score:0.9829222011385199
recall score:0.8131868131868132
f1 score:0.8900343642611684
roc_auc_score:0.9445861847550363
